In [1]:
import numpy as np

# Sensor de temperatura

## Expresión para la resistencia del NTC

$R_{NTC} = R_0 \cdot e^{\beta \cdot (\frac{1}{T} - \frac{1}{T_0})}$

In [2]:
def R_NTC(T0, R0, T):
    # T0 y T se ingresan en grados centígrados
    # R0 y R_NTC se consideran en ohms
    # beta está extraida de la hoja de datos
    
    T0 = 273 + T0
    T = 273 + T
    beta = 3434
    
    return R0 * np.exp(beta * (1/T - 1/T0))

Valores del NTC a temperaturas seleccionadas:

In [3]:
R35 = R_NTC(25, 10000, 35)
R60 = R_NTC(25, 10000, 60)
R85 = R_NTC(25, 10000, 85)

print(R35, R60, R85)

6878.81094646 2978.46322866 1449.58010402


## Cálculo de la resistencia de linealización (método de los 3 puntos)

In [4]:
def Rlin(RT1, RT2, RT3):
    num = RT2*(RT1+RT3)-2*RT1*RT3
    den = RT1+RT3-2*RT2
    return num/den

In [5]:
Rlinealizacion = Rlin(R35, R60, R85)
Rlinealizacion

2050.6447944614793

## Cálculo de las resistencias resultantes

In [6]:
def paralelo(R1, R2):
    Rpar = R1*R2/(R1+R2)
    return Rpar

In [7]:
R35lin = paralelo(R35, Rlinealizacion)
R60lin = paralelo(R60, Rlinealizacion)
R85lin = paralelo(R85, Rlinealizacion)

El NTC con la resistencia de linealización en paralelo, tendrá los siguientes valores a las temperaturas de 35, 60 y 85 grados centígrados.

In [8]:
print(R35lin, R60lin, R85lin)

1579.71530054 1214.48377869 849.252256833


### Verificación

La diferencia de resistencia entre 85 y 60 grados, debe ser igual que la diferencia de resistencia entre 60 y 35 grados.

In [9]:
R85lin - R60lin

-365.23152185415199

In [10]:
R60lin - R35lin

-365.23152185415188

## Obtención de una expresión para la recta del NTC linealizado

Como el NTC está linealizado por el método de los 3 puntos, la resistencia evaluada a esas 3 temperaturas deben quedar ubicadas sobre una recta, que se puede expresar como:

$R = R_{ref} \cdot (1+x)$ donde reemplazamos $x = -b \cdot (T-T_{ref})$


$R = R_{ref} \cdot (1-b \cdot (T-T_{ref}))$

$R = R_{ref} - R_{ref} \cdot b \cdot (T-T_{ref})$

donde $R_{ref}$ es la resistencia del NTC linealizado, a la temperatura $T_{ref}$, que en este caso se elige de $35$ grados (y es diferente a la temperatura nominal $T_0 = 25$ grados del NTC).

### Obtención del coeficiente b

$b = \frac{R_{ref} - R}{R_{ref} \cdot (T - T_{ref})}$

In [11]:
def b(Tref, Rref, T, R):
    # Tref y T en grados centígrados o en kelvin, es indistinto porque es un delta
    return (Rref-R)/(Rref*(T-Tref))

Como se dijo antes, la referencia es 35 grados, para la cual la resistencia es R35lin.

In [12]:
b(35, R35lin, 60, R60lin)

0.0092480340407916136

In [13]:
b(35, R35lin, 85, R85lin)

0.0092480340407916153

Asignamos el valor a una variable para poder usarlo después:

In [14]:
coef_b = b(35, R35lin, 85, R85lin)
coef_b

0.0092480340407916153

Podemos encontrar el paralelo del NTC con su resistencia de linealización,  a cualquier temperatura, mediante la siguiente función:

In [15]:
def res_aprox(Tref, Rref, b, T):
    # con Tref y T en grados centígrados
    
    return  Rref * (1 - b * (T-Tref))

In [16]:
R35aprox = res_aprox(35, R35lin, coef_b, 35)
R60aprox = res_aprox(35, R35lin, coef_b, 60)
R85aprox = res_aprox(35, R35lin, coef_b, 85)
print(R35aprox, R60aprox, R85aprox)

1579.71530054 1214.48377869 849.252256833


## Apartamiento de la linealidad

Evaluación de la diferencia entre ambas expresiones (valor real y valor aproximado de Rparalelo) en uno de los 3 puntos usados para determinar la recta aproximada:

In [17]:
print(R35lin - R35aprox, R60lin - R60aprox, R85lin - R85aprox)

0.0 0.0 0.0


Con el resultado anterior verificamos que esta recta representa el comportamiento lineal aproximado del paralelo entre el NTC y la resistencia de linealización. **¿Pero qué pasa fuera de estos 3 puntos?** Los valores de la resistencia linealizada se alejan un poco de la recta aproximada. Por ejemplo, a 70 grados centígrados, el valor que mediríamos con un ohmetro en condiciones ideales es:

In [18]:
R70lin = paralelo(R_NTC(25, 10000, 70), Rlinealizacion)
R70lin

1062.5266383707847

La resistencia estimada, de acuerdo con la linealización es distinta.

In [19]:
res_aprox(35, R35lin, coef_b, 70)

1068.3911699455714

Si encontramos la máxima diferencia (que sabemos que está entre medio de los puntos utilizados ya que la aproximación es sigmoidea), podremos encontrar la incertidumbre por no-linealidad. Por lo tanto, **las temperaturas a las cuales se producirá el mayor error será 35+(60-35)/2 = 47.5 y 60+(85-60)/2 = 72.5**. Si hubiéramos calculado la linealización por el punto de inflexión, sitúandolo en el medio del rango de medida (a 60 grados), el máximo error de no-linealidad debería calcularse en los puntos más alejados de éste, es decir a 35 y 85 grados. **Más adelante se calculan las incertidumbres asociadas a estos apartamientos de la recta teórica**.

# Rango (del sensor linealizado)

De acuerdo con la función de transferencia del flujímetro ($v = k \Delta T$, con $ k=0,04 \frac{m}{s \, ^{\circ}C}$), cuando la velocidad del flujo sea 0.2 m/s la diferencia de temperaturas entre los dos sensores será de $\Delta T = 5$. Cuando la velocidad del flujo sea 2 m/s la diferencia de temperaturas entre los dos sensores será de $\Delta T = 50$. 

El sensor 1 siempre estará a la misma temperatura de 35 grados centígrados (que es la temperatura de la incubadora, a la cual estará el flujo de aire antes de ser calentado), mientras que el sensor 2 estará a una temperatura de 35+5=40 cuando el flujo sea de 0.2 m/s y de 35+50=85 cuando el flujo sea de 2 m/s. Por ende, **los sensores deben ser capaces de medir en el rango de 35 a 85 grados centígrados**.

# Incertidumbre

La función de transferencia teórica del sistema permite convertir la resistencia leida en un sensor, en un valor de temperatura. Con los dos valores de temperatura así obtenidos, para el sensor 1 y el sensor 2, se obtiene el valor del flujo.

Debemos despejar $T=f(R)$ para saber cómo convertir la lectura de resistencia en temperatura. Anteriormente expresamos la recta teórica del sistema como:

$R = R_{ref} - R_{ref} \cdot b \cdot (T-T_{ref})$

Despejamos entonces T (recordemos que $T_{ref}$ era 35 grados centígrados y $R_{ref}$ era R35lin):

$ T = T_{ref} + \frac{1}{b}\cdot (1-\frac{R}{R_{ref}})$

A este valor de T vamos a llamarlo $T_{aprox}$ ya que es la temperatura aproximada considerando que el paralelo (entre el NTC y una resistencia de linealización) es lineal.

In [20]:
def T_aprox(Tref, Rref, b, R):
    # Tref y Taprox ambos en grados centígrados, o ambos en Kelvin
    # R en Ohms
    return Tref + (1/b) * (1-(R/Rref))

Recordemos el valor del coeficiente b, calculado anteriormente:

In [21]:
coef_b

0.0092480340407916153

A una temperatura de 47.5 grados, el valor del NTC y del paralelo, serán:

In [22]:
R475 = R_NTC(25, 10000, 47.5)
R475

4453.1145064120046

In [23]:
R475lin = paralelo(R475, Rlinealizacion)
R475lin

1404.0734995356056

La temperatura que será leida por el sistema, cuando la temperatura sea de 47.5 grados será:

In [24]:
T_aprox(35, R35lin, coef_b, R475lin)

47.022634308376979

**Vemos que habrá un error de poco menos de medio grado (cuando haya 47.5 sobre el sensor 2, el sistema estimará que ese sensor está midiendo 47.0). Como la máxima incertidumbre admisible es $\pm 0.1 m/s$, de acuerdo a la F de T del sistema equivale a un $\Delta T$ de 2.5 grados centígrados, y por lo tanto el sensor es apto para esta aplicación.**

## Incertidumbre debida a la tolerancia en la resistencia nominal

La incertidumbre en la medida puede tener relación con las tolerancias de los parámetros del NTC (resistencia nominal y beta), y con la constante de disipación térmica (ya que si el NTC se calienta por encima de la temperatura que mide, estará arrojando un valor incorrecto). Esta incertidumbre es más dificil de estimar porque lo que genera es un error en el cálculo de la resistencia de linealización y en la obtención de la recta aproximada.

Por otra parte, estas incertidumbres se pueden neutralizar durante el proceso de ajuste previo a la calibración, en el cual se fija la salida del sistema para los extremos del rango y se corrigen el offset y ganancia, de modo que la recta aproximada pasa exactamente por donde esperábamos que pase. Por este motivo, no sería correcto considerar esto como una incertidumbre en la medida, pero lo calculamos igual como *ejercicio del pensamiento*. 

Estimaremos cuál sería la temperatura que leería el flujímetro térmico si no se realizara ningún ajuste, y el sensor linealizado funcionase según la recta aproximada que se calculó antes, siendo posible que su resistencia nominal a 25 grados esté entre los extremos de $10K\Omega \pm 1\%$ es decir entre 9.9K y 10.1K.

Para simplificar, sólo analizaremos lo que sucede en un solo punto del rango de entrada (85 grados centígrados en el sensor 2). Los valores posibles para el NTC en este rango de resistencias nominales, pero a 85 grados centígrados sería:

In [25]:
R85_a=R_NTC(25, 9900, 85)
R85_b=R_NTC(25, 10000, 85)
R85_c=R_NTC(25, 10100, 85)

Recordemos el valor de la resistencia de linealización:

In [26]:
Rlinealizacion

2050.6447944614793

Los valores del paralelo serán:

In [27]:
R85lin_a = paralelo(R85_a, Rlinealizacion)
R85lin_b = paralelo(R85_b, Rlinealizacion)
R85lin_c = paralelo(R85_c, Rlinealizacion)
print(R85lin_a, R85lin_b, R85lin_c)

844.256129277 849.252256833 854.207173222


La temperatura leida para los 3 valores diferentes de resistencia nominal serían:

In [28]:
T85a = T_aprox(35, R35lin, coef_b, R85lin_a)
T85b = T_aprox(35, R35lin, coef_b, R85lin_b)
T85c = T_aprox(35, R35lin, coef_b, R85lin_c)
print(T85a, T85b, T85c)

85.3419835951 85.0 84.6608372982


Es decir que la variación dentro del $1 \%$ de la resistencia nominal del NTC puede producir una incerteza de $\pm 0.35$ grados centígrados, por debajo de los $2.5$ grados centígrados admisibles en la incertidumbre. Un cálculo semejante podríamos hacer considerando la variación en el beta, aunque ambos se podrían corregir durante el ajuste previo a la calibración.

# Sensor de presión

## Rango

La función de transferencia para convertir la diferencia presión medida en velocidad es:
    
$v = c \cdot \sqrt{a \cdot \Delta P}$

donde c = 0.04 y $a = 7.5 \frac{m^4}{s^2 \cdot kgf}$

Para evaluar el rango de diferencia de presiones requerido para el rango esperado de entrada, podemos despejar $\Delta P$:

$\Delta P = \frac{1}{a} \cdot \left( \frac{v}{c} \right)^2$

Cabe recordar que $1 kgf = 1 kg \cdot 9.8 \frac{m}{s^2} = 9.8 \frac{kg \cdot m}{s^2}$ y por lo tanto se puede expresar $a = \frac{7.5}{9.8} \frac{m^4 \cdot s^2}{s^2 \cdot kg \cdot m }$ y simplificando $a = \frac{7.5}{9.8} \frac{m^3}{kg}$.

### Extremos del rango

In [29]:
def deltaP(v):
    # v expresada en m/s
    # deltaP expresada en Pascales
    return 9.8/7.5 * (v/0.04)**2

deltaP(0.2)

32.66666666666667

In [30]:
deltaP(2)

3266.666666666667

### Verificación de las unidades

Para verificar las unidades, se muestra a continuación el desarrollo para $v = 0.2 \frac{m}{s}$:

$\Delta P = \frac{9.8}{7.5} \frac{Kg}{m^3} \cdot \left( \frac{0.2 \frac{m}{s}}{0.04} \right)^2$

$\Delta P = \frac{9.8}{7.5} \cdot 5^2 \cdot \frac{kg \cdot m^2}{m^3 \cdot s^2}$

$\Delta P = 32.66 \frac{kg}{m \cdot s^2} = 32.66 \, Pa$

Por lo tanto el sensor debe ser capaz de medir en un rango que vaya desde 32.66 Pa hasta por lo menos 3266.67 Pa.

### Conversión a PSI

Convertimos los extremos del rango y la incertidumbre a PSI, para evaluar la entrada requerida en el sensor. Sabemos que 1 PSI = 6894,76 Pa.

In [31]:
def pa2psi(pa):
    return pa / 6894.76

In [32]:
pa2psi(32.66)

0.00473693065458406

In [33]:
pa2psi(3266.67)

0.473790240704535

Por lo tanto puede utilizarse el sensor cuyo rango de entrada está entre 0 y 1 PSI.

## Incertidumbre

El requerimiento es que se pueda medir la velocidad del flujo con una incertidumbre menor a 0.1 m/s. Si convertimos esto a presión tendremos:

In [34]:
deltaP(0.1)

8.166666666666668

In [35]:
pa2psi(8.17)

0.0011849578520499626

Es decir que el sensor debe provocar como máximo un error de 8.1 Pa = 0.0012 PSI.

El máximo error por no-linealidad e histéresis es del 0.5% de fondo de escala (1 PSI), es decir: $0.5 / 100 \cdot 1 PSI = 0.005 PSI$, lo cual está por encima de la máxima incertidumbre admisible, con lo cual el sensor no es apto para esta aplicación.

## Conversión de diferencia de presión en velocidad de flujo

In [36]:
def vFlujo(deltaP):
    # deltaP debe estar expresado en Pascales
    # el resultado se da en m/s
    c = 0.04
    a = 7.5/9.8
    raiz = np.sqrt(a * deltaP)
    return c * raiz

In [37]:
vFlujo(32.66)

0.1999795907953956

In [38]:
vFlujo(3266.67)

2.0000010204079031